In [3]:
#!pip install ohmysportsfeedspy
#!pip install simplejson

  Stored in directory: /Users/zspahr/Library/Caches/pip/wheels/31/e7/31/0ff3af975cb02b5096d797449c1552a46645c01dbf0501ef0c
Successfully built ohmysportsfeedspy


In [4]:
from ohmysportsfeedspy import MySportsFeeds
import simplejson as json
import pandas as pd




In [24]:
msf = MySportsFeeds(version="1.2")
msf.authenticate("ff3a9d40-4a07-42fb-91f6-00446c","6dRSvQq4F3evp!C")


In [26]:
#Available stats for each game by a team
#Testing connection
TeamGameLogs = msf.msf_get_data(league='nhl',season="2016-2017-regular",feed='team_gamelogs',team=5, format='json')
try:
    TeamGameLogs["teamgamelogs"]["gamelogs"][0]["stats"]
except KeyError:
    print ("Connection error")
    



Connection error


In [ ]:
## Building a loop to go through each season,each team, each game
## What is missing is the full list of team IDS, and including a Try and Except.
## we can use this lopp to build the dataframe of the stats we need.
##This is a limited test over three seasons

#Defining Lists

ListSeason=[]
ListTeam=[]
ListblockedShots=[]
ListgoalsAgainst=[]
ListpenaltyKillPercent=[]
ListWin=[]
ListShorthandedGoalsAgainst=[]


#Defining seasons

TargetSeasons=["2018-2019-regular",
              "2017-2018-regular",
              "2016-2017-regular"]

#Creating loops to fill the lists
try:
    for season in TargetSeasons:
        for team in range (1,33):
            for game in range (1,83):
                try:
                    TeamGameLogs = msf.msf_get_data(league='nhl',season=season,feed='team_gamelogs',team=team, format='json')
                    BlockedShots=TeamGameLogs["teamgamelogs"]["gamelogs"][game]["stats"]["BlockedShots"]["#text"]
                    win=TeamGameLogs["teamgamelogs"]["gamelogs"][game]["stats"]["Wins"]["#text"]
                    goalsAgainst=TeamGameLogs["teamgamelogs"]["gamelogs"][game]["stats"]["GoalsAgainst"]["#text"]
                    PenaltyKillPercent=TeamGameLogs["teamgamelogs"]["gamelogs"][game]["stats"]["PenaltyKillPercent"]["#text"]
                    ShorthandedGoalsAgainst=TeamGameLogs["teamgamelogs"]["gamelogs"][game]["stats"]["ShorthandedGoalsAgainst"]["#text"]

                except:
                    continue

                ## Adding values to list
                ListSeason.append(season)
                ListTeam.append(team)
                ListblockedShots.append(BlockedShots)
                ListgoalsAgainst.append(goalsAgainst)
                ListWin.append(win)
                ListpenaltyKillPercent.append(PenaltyKillPercent)
                ListShorthandedGoalsAgainst.append(ShorthandedGoalsAgainst)
                print("checkpoint")
except:
    print("Error Found")
    
    
print ("Completed")

In [ ]:
#zipping the lists and creating a dataframe

ZipList=list(zip(ListSeason, ListTeam,ListblockedShots,ListgoalsAgainst,ListpenaltyKillPercent,ListWin,ListShorthandedGoalsAgainst))

Gamesdf = pd.DataFrame(ZipList, columns = ["Season",
                                           "Team",
                                           "BlockedShots",
                                           "Goals Against",
                                           "Penalty Kill Percent",
                                           "Win",
                                           "Shorthanded Goals Allowed"])

Gamesdf["BlockedShots"]=Gamesdf["BlockedShots"].astype('int32')
Gamesdf["Goals Against"]=Gamesdf["Goals Against"].astype('int32')
Gamesdf["Shorthanded Goals Allowed"]=Gamesdf["Shorthanded Goals Allowed"].astype('int32')

Gamesdf["Penalty Kill Percent"]=Gamesdf["Penalty Kill Percent"].astype('float')


In [ ]:
#Grouping by win/loss to compare stats

WinLoss=Gamesdf.groupby("Win").mean()
WinLoss=WinLoss.drop(['Team'], axis=1)
WinLoss

In [ ]:
#available feeds

['cumulative_player_stats',
 'full_game_schedule',
 'daily_game_schedule',
 'daily_player_stats',
 'game_boxscore',
 'scoreboard',
 'game_playbyplay', 
 'player_gamelogs',
 'team_gamelogs',
 'roster_players',
 'game_startinglineup',
 'active_players',
 'overall_team_standings',
 'conference_team_standings',
 'division_team_standings',
 'playoff_team_standings',
 'player_injuries',
 'daily_dfs',
 'current_season',
 'latest_updates']